In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.constraints import maxnorm
from keras.optimizers import adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.
/Users/jackholder/anaconda2/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
np.random.seed(19)

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [31]:
import tensorflow as tf
from sklearn.utils.random import sample_without_replacement
from sklearn.utils import check_random_state
from scipy.linalg import circulant
from scipy.linalg import hadamard
from scipy.linalg import toeplitz
import random
import math
import tqdm

def gaussian_random_tensor(n_components, n_features, random_state=None):
    rng = check_random_state(random_state)
    components = rng.normal(loc=0.0,
                            scale=1.0 / np.sqrt(n_components),
                            size=(n_components, n_features))
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def hadamard_mat(n_components, n_features):
    had = hadamard(n_features, dtype=complex)
    had = had[np.random.choice(had.shape[0], n_components, replace=False)]
    return had

def diagonal_mat(n_components, n_features):
    components = np.random.rand(n_features)
    components = np.diag(components)
    components = components[0:n_components]
    components = np.sign(components)
    return components

def circulant_random_tensor(n_components, n_features):
    components = np.random.normal(0, 1, (1, n_features))
    components = circulant(components)
    diag1 = diagonal_mat(n_features, n_features)
    diag2 = diagonal_mat(n_features, n_features)
    had = hadamard_mat(n_features, n_features)
    HD = np.dot(had, diag1)
    DHD = np.dot(diag2, HD)
    components = np.dot(components, DHD) # Extended Ψ-regular hashing
    # components = np.dot(components, diag1) # Short Ψ-regular hashing 
    components = components[np.random.choice(components.shape[0], n_components, replace=False)]
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def toeplitz_random_tensor(n_components, n_features):
    components = np.random.normal(0, 1, (1, n_features))
    components = toeplitz(components)
    diag1 = diagonal_mat(n_features, n_features)
    diag2 = diagonal_mat(n_features, n_features)
    had = hadamard_mat(n_features, n_features)
    HD = np.dot(had, diag1)
    DHD = np.dot(diag2, HD)
    components = np.dot(components, DHD) # Extended Ψ-regular hashing
    # components = np.dot(components, diag1) # Short Ψ-regular hashing 
    components = components[np.random.choice(components.shape[0], n_components, replace=False)]
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def kac_matrix(dim, n):
    maxiter = n * math.log(n)
    for k in tqdm.tqdm(range(1, int(maxiter))):
        i = random.randint(0, n-1)
        j = random.randint(0, n-1)
        while j == i:
            j = random.randint(0, n-1)
        theta = np.random.uniform(0, 2*math.pi)
        B_new = np.identity(n)
        B_new[i, i] = math.sin(theta)
        B_new[i, j] = math.cos(theta)
        B_new[j, i] = -math.cos(theta)
        B_new[j, j] = math.sin(theta)
        if k == 1:
            M = B_new
        else:
            M = np.dot(M, B_new)
    M = M[np.random.choice(M.shape[0], dim, replace=False)]
    return (tf.convert_to_tensor(M, dtype = 'float32'))
          
def project(x, Y):  
    features = K.int_shape(x)[1] 
    x = tf.transpose(x)
    # Y = tf.sign(Y) # use this if it is doing the sign BEFORE (we think right, he supposedly said wrong)
    X_new = K.dot(Y, x)
    X_new = tf.transpose(X_new)
    # X_new = tf.sign(X_new) # use this if it is doing the sign AFTER (we think wrong, he says right)
    return X_new

In [ ]:
dims = [1024, 512, 256, 128, 64]

import time
accuracy = []
size = []
times = []
epochs = 10

for d in dims:
    start = time.time()
    
    if d != 2048:
        # Y = gaussian_random_tensor(d, 2048) 
        # Y = kac_matrix(d, 2048) 
        # Y = circulant_random_tensor(d, 2048) 
        Y = toeplitz_random_tensor(d, 2048)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    if d != 2048:
        model.add(Lambda (lambda x: project(x, Y)))
        
    model.add(Dense(50, activation='relu', kernel_constraint=maxnorm(3)))
    
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    h = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
    a = model.evaluate(X_test, y_test, verbose=0)[1]
    
    end = time.time()
    t = end - start
    accuracy.append(a)
    size.append(d)
    times.append(t)

import pandas as pd
df = pd.DataFrame({'Size': size, 'Accuracy': accuracy, 'Runtime': times})

/Users/jackholder/anaconda2/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:355: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
 3136/50000 [>.............................] - ETA: 286s - loss: 14.4940 - acc: 0.1008

In [27]:
df['time_saved'] = (max(df['Runtime']) - df['Runtime']) / max(df['Runtime'])
df['acc_lost'] = (max(df['Accuracy']) - df['Accuracy'])
df

,Accuracy,Runtime,Size,time_saved,acc_lost
0,0.6421,3125.488419,1024,0.000000,0.0167
1,0.6588,2842.399565,512,0.090574,0.0000
2,0.6344,2790.572193,256,0.107156,0.0244
3,0.6146,2771.782638,128,0.113168,0.0442
4,0.6043,2771.019614,64,0.113412,0.0545


In [28]:
df.to_csv('Cifar_circ_nosign.csv')

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 32, 16, 16)        9248      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 32, 8, 8)          0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 2048)              0         
_________________________________________________________________
lambda_25 (Lambda)           (None, 64)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 50)                3250      
__________

In [ ]:
model.get_layer('dense_156')

In [ ]:
from keras.models import Model
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('dense_156').output)
intermediate_output = intermediate_layer_model.predict(X_test)

In [ ]:
from sklearn.manifold import TSNE
t_sne = TSNE(n_components=2, random_state=0)
ts = t_sne.fit_transform(intermediate_output)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

def scatter(x, colors):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

scatter(ts, y_test)